## Read and Rename

In [90]:
import pandas as pd
import sys
sys.path.append('../')
from utils.uuid_util import generate_uuid

education = pd.read_csv('../data/Cleaned/education/education.csv')
employment = pd.read_csv('../data/Cleaned/employment/european_employment_data.csv')
socioecono = pd.read_csv('../data/Cleaned/socio/socio.csv')

education.rename(columns={'Country Name': 'country_name',
                          'Country Code': 'country_code',
                          'Indicator Name': 'indicator_name',
                          'Value': 'value',
                          }, inplace=True)
employment.rename(columns={'Country Name': 'country_name',
                           'Country Code': 'country_code',
                            'Indicator Name': 'indicator_name',
                            '2023': 'unemployment_rate',
                            }, inplace=True)
employment.drop(columns=['Unnamed: 0'], inplace=True)
sociecono = socioecono.rename(columns={'country': 'country_name',
                                       'wbid': 'country_code',
                                       'class': 'socioeconomic_class',
                                        }, inplace=True)

## Indicator Selection

In [91]:
# Government expenditure on education as % of GDP (%),SE.XPD.TOTL.GD.ZS
# Government expenditure on secondary education as % of GDP (%),UIS.XGDP.23.FSGOV
# Government expenditure on tertiary education as % of GDP (%),UIS.XGDP.56.FSGOV


categories_dict = {
    "Government Expenditure": [
    "SE.XPD.TOTL.GD.ZS",
    "UIS.XGDP.23.FSGOV",
    "UIS.XGDP.56.FSGOV",
    ],
    "Gender Equality": [
        "SE.TER.CMPL.ZS",
    ]     
}



categories_reverse = {}
for key in categories_dict.keys():
    for item in categories_dict[key]:
        categories_reverse[item] = key
        
categories_reverse

{'SE.XPD.TOTL.GD.ZS': 'Government Expenditure',
 'UIS.XGDP.23.FSGOV': 'Government Expenditure',
 'UIS.XGDP.56.FSGOV': 'Government Expenditure',
 'SE.TER.CMPL.ZS': 'Gender Equality'}

In [92]:
selected_indicators_code = list(categories_reverse.keys())

## Export countries

In [93]:
employment.head()

,country_name,country_code,indicator_name,Indicator Code,unemployment_rate
0,Austria,AUT,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,5.268
1,Belgium,BEL,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,5.541
2,Bulgaria,BGR,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,4.185
3,Switzerland,CHE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,3.838
4,Germany,DEU,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,3.006


In [94]:
# export country nodes
country = employment[['country_name', 'country_code']].copy(deep=True)
# generate unique 3-digit country id
import uuid
country['country_id'] = country['country_code'].apply(lambda x: uuid.uuid3(uuid.NAMESPACE_DNS, x))
country.to_csv(f'../data/neo4j/country.csv', index=False)


## Export education nodes

In [95]:
# add indicator code
edu_raw = pd.read_csv('../data/European/EducationStats/european_education_data.csv')

edu_raw_index = edu_raw[['Indicator Name', 'Indicator Code']].drop_duplicates()
edu_raw_index.rename(columns={'Indicator Name': 'indicator_name',
                                'Indicator Code': 'indicator_code'}, inplace=True)

# add education indicator code to our file

education = pd.merge(education, edu_raw_index, on='indicator_name', how='left')
# education.to_csv(f'../data/neo4j/education.csv', index=False)
indeces = education[['indicator_name', 'indicator_code']].drop_duplicates()
indeces.to_csv(f'../data/neo4j/index.csv', index=False)

In [83]:
# add UUID to every row
education['educational_index_id'] = education.apply(lambda row: generate_uuid(), axis=1)

In [84]:
# export education nodes
educational_index_fields = ['indicator_name', 'value', 'indicator_code', 'educational_index_id']
educational_index = education[educational_index_fields]

educational_index_selected = educational_index[educational_index['indicator_code'].isin(selected_indicators_code)]
educational_index_selected.to_csv(f'../data/neo4j/educational_index.csv', index=False)

## Categories

In [85]:
# "Gross graduation ratio from first degree programmes (ISCED 6 and 7) in tertiary education, both sexes (%)",SE.TER.CMPL.ZS
# "Gross graduation ratio from first degree programmes (ISCED 6 and 7) in tertiary education, female (%)",SE.TER.CMPL.FE.ZS
# "Gross graduation ratio from first degree programmes (ISCED 6 and 7) in tertiary education, gender parity index (GPI)",UIS.GGR.5.A.GPI
# "Gross graduation ratio from first degree programmes (ISCED 6 and 7) in tertiary education, male (%)",SE.TER.CMPL.MA.ZS

categories = pd.DataFrame({
    'indicator_code': list(categories_reverse.keys()),
    'category': list(categories_reverse.values()),
})\
    .merge(indeces, on='indicator_code', how='left')[['category', 'indicator_name', 'indicator_code']]\
    .to_csv(f'../data/neo4j/categories.csv', index=False)

## Education Index - Country relationship

In [86]:
education_country = education[['country_name','indicator_code', 'country_code', 'educational_index_id']].copy(deep=True)
education_country['country_id'] = pd.merge(education_country, country, on='country_name', how='left')['country_id']
education_country.to_csv(f'../data/neo4j/education_country.csv', index=False)

## Social Economic Class

In [87]:
# add UUID to every "row"
socioecono['socioeconomic_id'] = socioecono.apply(lambda row: generate_uuid(), axis=1)
socioecono.to_csv(f'../data/neo4j/socioecono.csv', index=False)
socioecono.head()

,unid,country_code,country_name,year,ses,socioeconomic_class,gdppc,yrseduc,region5,regionUN,socioeconomic_id
0,100,BGR,Bulgaria,2000,64.527023,Middle(semi-per),8958.050781,9.3762,SE Europe,East Europe,087cce52-822c-4813-91c2-dc788f06c9f5
1,100,BGR,Bulgaria,2010,63.851353,Middle(semi-per),15283.179690,9.8510,SE Europe,East Europe,452dc637-51c2-46fe-9305-3436afdb915d
2,208,DNK,Denmark,2000,86.824326,High(core),42337.714840,9.7124,NW Europe,North Europe,f0eb912b-832a-4284-8246-bc8f64e205e1
3,208,DNK,Denmark,2010,82.432434,High(core),43998.437500,10.0566,NW Europe,North Europe,14da39a9-bc75-42ab-aae1-5aa4f4bfaed4
4,246,FIN,Finland,2000,70.608109,High(core),34887.179690,8.1934,NW Europe,North Europe,2dcf2b92-4bc6-4a03-b28e-ea3694b9330b


# Employment

In [88]:
## Employment Rate
# add uuid to every row
employment['employment_id'] = employment.apply(lambda row: generate_uuid(), axis=1)
employment.to_csv(f'../data/neo4j/employment.csv', index=False)
employment.head()

,country_name,country_code,indicator_name,Indicator Code,unemployment_rate,employment_id
0,Austria,AUT,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,5.268,2ac4ae56-33b4-4bba-8dfe-e622e7744cbd
1,Belgium,BEL,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,5.541,21895466-2068-4d24-98f3-ba785ba95144
2,Bulgaria,BGR,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,4.185,3a331d5c-9859-4fa2-8fe6-1a2078fd29cf
3,Switzerland,CHE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,3.838,c91bae40-d491-4afc-b4b2-54ed7644eae8
4,Germany,DEU,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,3.006,7d919876-f20a-47d3-837e-30e8d74c7fc4


# Correlation